In [3]:

from xgboost import XGBClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from fraud import collectAllFeaturesBaseline


df_fraud_aggregated = collectAllFeaturesBaseline() 

/Users/muri/education/spiced/WaterMLOps/ds-mlproject-fraud/fraud.py:58: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  self._frames[str(p)] = pd.read_csv(p)


           client_id  target  disrict  region  client_catg
0     train_Client_0     0.0       60     101           11
1     train_Client_1     0.0       69     107           11
2    train_Client_10     0.0       62     301           11
3   train_Client_100     0.0       69     105           11
4  train_Client_1000     0.0       62     303           11


In [4]:
# ---------------------------------------------------------------------
# Beispiel: BalancedBaggingClassifier (imblearn) mit XGBClassifier
# ---------------------------------------------------------------------

# 1) DataFrame laden / bereitstellen
#    Ersetze das hier durch dein echtes df_fraud_aggregated
df = df_fraud_aggregated  

# 2) Features und Target definieren
X = df.drop(columns=["client_id", "target"]).fillna(0)
y = df["target"]

# 3) Train/Test-Split (stratifiziert)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    stratify=y,
    random_state=42
)

# 4) Basis-XGBoost-Modell (wichtigste Hyperparameter)
xgb_base = XGBClassifier(
    use_label_encoder=False,
    eval_metric="logloss",
    n_estimators=150,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

# 5) Imbalanced-learn Ensemble mit BalancedBagging
bbc = BalancedBaggingClassifier(
    estimator=xgb_base,
    sampling_strategy="auto",   # Minority-Klasse auf Majorität hochziehen
    n_estimators=10,            # Anzahl der Bagging-Modelle
    random_state=42,
    replacement=False           # ohne Zurücklegen
)

# 6) Training
bbc.fit(X_train, y_train)

# 7) Vorhersage & Evaluation
y_pred = bbc.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

/Users/muri/education/spiced/WaterMLOps/ds-mlproject-fraud/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:15:16] WARNING: /private/var/folders/49/1y5t6tjd2hxgddgr4h8l86lr0000gn/T/pip-install-xwm7t153/xgboost_f76b7daff02d438b84bd67ed3f83a334/cpp_src/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/muri/education/spiced/WaterMLOps/ds-mlproject-fraud/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:15:16] WARNING: /private/var/folders/49/1y5t6tjd2hxgddgr4h8l86lr0000gn/T/pip-install-xwm7t153/xgboost_f76b7daff02d438b84bd67ed3f83a334/cpp_src/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/Users/muri/education/spiced/WaterMLOps/ds-mlproject-fraud/.venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [16:15:16] WARNING: /private/var/folders/49/1y5t6tjd2hxgddgr4h8l86lr0000gn/T/p

Confusion Matrix:
[[27026 11352]
 [  769  1501]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.97      0.70      0.82     38378
         1.0       0.12      0.66      0.20      2270

    accuracy                           0.70     40648
   macro avg       0.54      0.68      0.51     40648
weighted avg       0.92      0.70      0.78     40648

